# 1. Getting the data

### The main code for querying the Places API
After running this the object response will have the json we queried stored in pure text format.\
Some adjustments will be needed to get more than 20 items from the query, we can do this by using nextPageToken.

In [52]:
import requests, json
from secretfile import api_key

origin_location = "57.70867542111475,11.973303298399806"
radius_in_meters = 1000
query_string = "burger"

# For later use...
restaurant_type = ["burger", "pizza", "thai", "italian", "chinese", "sushi", "grill", "taco", "vietnamese", "tapas", "kebab", "café", "salad", "indian", "poke"]

# The token we use for nextpage
next_page_token = ""

# The base url which we base our searches on
url_textsearch = f"https://maps.googleapis.com/maps/api/place/textsearch/json?query={query_string}&type=restaurant&location={origin_location}&radius={radius_in_meters}&key={api_key}&pagetoken={nextpage_token}"

# The url we use for going to the next page of searches
url_nextpage = f"https://maps.googleapis.com/maps/api/place/textsearch/json?key={api_key}&pagetoken={next_page_token}"

# Payload and headers, empty for this query
payload={}
headers = {}

# The keys function as attributes in this dict
result_dict = {"name": [], "rating": [], "user_ratings_total": [], "formatted_address": [], "place_id": []}

# The keys function as entities in this dict


def get_request(url):
    response = requests.request("GET", url, headers=headers, data=payload)
    response.raise_for_status()
    return response


response_to_dict(get_request(url_textsearch))
response_to_dict(get_request(url_textsearch))

populate_openinghours()

### Querying for multiple types of restaurants

Will add code here for getting more results and from all types of restaurants

### Formatting the data we just got

We want to write our results to a file temporarily for convenience, then populate a dataframe with the data.\
We will use the PlaceID as a foreign key and create two different dataframes, one for the places themselves\
and one for their opening hours. For the restaurants that don't have opening hours values\
for a certain day we will want to have the value be "closed".

In [47]:
def response_to_dict(response):
    global next_page_token
    with open('restaurants.json', 'w' , encoding='utf-8') as w:
        w.write(response.text)
    # Populate the dict with values from the results
    with open('restaurants.json', 'r', encoding='utf-8') as data_file:
        data = json.load(data_file)
        for restaurant in data['results']:
            result_dict['place_id'].append(restaurant['place_id'])
            result_dict['name'].append(restaurant['name'])
            result_dict['rating'].append(restaurant['rating'])
            result_dict['user_ratings_total'].append(restaurant['user_ratings_total'])
            result_dict['formatted_address'].append(restaurant['formatted_address'])
    next_page_token = data['next_page_token']

There is still some data that we have not been able to catch; the opening hours\
We will need to iterate through the names in a Place Details query against the Places API

In [17]:
def populate_openinghours():
# We query for the opening hours of every restaurant
    for place_id in result_dict['place_id']:
        url_findplace = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=opening_hours&key={api_key}"
        response = get_request(url_findplace)
        findplace_data = json.loads(response.text)

        # Since the data we get out form the query

        if 'opening_hours' in findplace_data['result']:
            openinghours_dict[place_id] = findplace_data['result']['opening_hours']['weekday_text']
        else:
            openinghours_dict[place_id] = "None" 

We will need to clean the opening_hours later, for now we will keep them in that format.

Now that we have the data we need to put it in a dataframe


In [18]:
import pandas as pd

In [53]:
# Create a neat dataframe and sort the rows by rating, user_ratings_total
df_places = pd.DataFrame.from_dict(result_dict)
df_places.set_index('place_id')
df_places['rating'] = pd.to_numeric(df_places['rating'])
df_places['user_ratings_total'] = pd.to_numeric(df_places['user_ratings_total'])
df_places.sort_values(['rating', 'user_ratings_total'], ascending=False)

,name,rating,user_ratings_total,formatted_address,place_id
11,"DINÉ Burgers, femman huset",4.7,31,"Postgatan 26-31, 411 06 Göteborg",ChIJfXcB_27zT0YRn_cGsJgRclQ
31,"DINÉ Burgers, femman huset",4.7,31,"Postgatan 26-31, 411 06 Göteborg",ChIJfXcB_27zT0YRn_cGsJgRclQ
6,2112,4.5,1917,"Magasinsgatan 5, 411 18 Göteborg",ChIJPSytJGbzT0YRKUOpf_qtaeM
26,2112,4.5,1917,"Magasinsgatan 5, 411 18 Göteborg",ChIJPSytJGbzT0YRKUOpf_qtaeM
7,Master Burger,4.5,53,"Södra Vägen 30, 412 54 Göteborg",ChIJ2TwN_YvzT0YRBwhJIPkYUD8
27,Master Burger,4.5,53,"Södra Vägen 30, 412 54 Göteborg",ChIJ2TwN_YvzT0YRBwhJIPkYUD8
17,Johnny's Burger and Shakes,4.5,35,"Vasa Allen / avenue, Vasagatan 56, 411 36 Göte...",ChIJsfR44DXzT0YRcMekQlC1euw
37,Johnny's Burger and Shakes,4.5,35,"Vasa Allen / avenue, Vasagatan 56, 411 36 Göte...",ChIJsfR44DXzT0YRcMekQlC1euw
1,DINÉ Burgers,4.4,927,"Drottninggatan 31, 411 14 Göteborg",ChIJq2LaBCjzT0YRi-8wBQsPM3E
21,DINÉ Burgers,4.4,927,"Drottninggatan 31, 411 14 Göteborg",ChIJq2LaBCjzT0YRi-8wBQsPM3E


In [30]:
df_opening_hours = pd.DataFrame.from_dict(openinghours_dict, orient='index')
df_opening_hours

,0,1,2,3,4,5,6
ChIJsVTFD3DzT0YRH5WMvE6q5kQ,Monday: 11:30 AM – 12:00 AM,Tuesday: 11:30 AM – 12:00 AM,Wednesday: 11:30 AM – 12:00 AM,Thursday: 11:30 AM – 12:00 AM,Friday: 11:30 AM – 1:00 AM,Saturday: 12:00 PM – 1:00 AM,Sunday: 12:00 – 11:00 PM
ChIJq2LaBCjzT0YRi-8wBQsPM3E,Monday: 11:00 AM – 9:00 PM,Tuesday: 11:00 AM – 9:00 PM,Wednesday: 11:00 AM – 9:00 PM,Thursday: 11:00 AM – 9:00 PM,Friday: 11:00 AM – 10:00 PM,Saturday: 12:00 – 10:00 PM,Sunday: 12:00 – 7:00 PM
ChIJt2e12WXzT0YR1mzQqaFk6bc,Monday: 11:00 AM – 9:00 PM,Tuesday: 11:00 AM – 9:00 PM,Wednesday: 11:00 AM – 9:00 PM,Thursday: 11:00 AM – 9:00 PM,Friday: 11:00 AM – 10:00 PM,Saturday: 12:00 – 10:00 PM,Sunday: 12:00 – 8:00 PM
ChIJT_lXYH_zT0YRW104WmLMNzk,Monday: 11:00 AM – 9:00 PM,Tuesday: 11:00 AM – 9:00 PM,Wednesday: 11:00 AM – 9:00 PM,Thursday: 11:00 AM – 9:00 PM,Friday: 11:00 AM – 10:00 PM,Saturday: 12:00 – 10:00 PM,Sunday: 12:00 – 8:00 PM
ChIJ22Wom6_zT0YRroSDRonvRPc,Monday: 11:00 AM – 9:00 PM,Tuesday: 11:00 AM – 9:00 PM,Wednesday: 11:00 AM – 9:00 PM,Thursday: 11:00 AM – 9:00 PM,Friday: 12:00 – 10:00 PM,Saturday: 12:00 – 10:00 PM,Sunday: 12:00 – 8:00 PM
ChIJRZARJXzzT0YRvW3cV1T8eRw,Monday: 11:00 AM – 9:00 PM,Tuesday: 11:00 AM – 9:00 PM,Wednesday: 11:00 AM – 9:00 PM,Thursday: 11:00 AM – 9:00 PM,Friday: 11:00 AM – 9:00 PM,Saturday: 11:00 AM – 9:00 PM,Sunday: 11:00 AM – 9:00 PM
ChIJPSytJGbzT0YRKUOpf_qtaeM,Monday: 4:00 PM – 12:00 AM,Tuesday: 4:00 PM – 12:00 AM,Wednesday: 4:00 PM – 12:00 AM,Thursday: 4:00 PM – 12:00 AM,Friday: 4:00 PM – 2:00 AM,Saturday: 1:00 PM – 2:00 AM,Sunday: 4:00 PM – 12:00 AM
ChIJ-waSFX7zT0YR0B_BWNeYf78,Monday: 11:00 AM – 1:30 PM,"Tuesday: 11:00 AM – 1:30 PM, 4:30 – 9:00 PM","Wednesday: 11:00 AM – 1:30 PM, 4:30 – 9:00 PM","Thursday: 11:00 AM – 1:30 PM, 4:30 – 9:00 PM","Friday: 11:00 AM – 1:30 PM, 4:30 – 9:30 PM",Saturday: 9:00 AM – 9:30 PM,Sunday: 9:00 AM – 8:30 PM
ChIJfXcB_27zT0YRn_cGsJgRclQ,Monday: 11:00 AM – 4:00 PM,Tuesday: 11:00 AM – 4:00 PM,Wednesday: 11:00 AM – 4:00 PM,Thursday: 11:00 AM – 4:00 PM,Friday: 11:00 AM – 4:00 PM,Saturday: 12:00 – 5:00 PM,Sunday: Closed
ChIJ2z6fanDzT0YRmAxGTu1WzKM,Monday: 10:00 AM – 11:00 PM,Tuesday: 10:00 AM – 11:00 PM,Wednesday: 10:00 AM – 11:00 PM,Thursday: 10:00 AM – 11:00 PM,Friday: 10:00 AM – 3:00 AM,Saturday: 10:00 AM – 3:00 AM,Sunday: 10:00 AM – 11:00 PM


### Storing the data